In [1]:
import plotly.express as px
import time
import pandas as pd
import numpy as np

# Data importing and Cleaning

In [2]:
df=pd.read_csv("Data_Age_Cleaned.csv",index_col="RA_Report #")

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#There are NaNs in symptoms, we need to rename them into None
df["SYM_One Row Coded Symptoms"].fillna(value="No Symptoms",inplace=True)
df["Category"].fillna(value="Not defined",inplace=True)
#There are leading spaces... We need to omit them
df["SYM_One Row Coded Symptoms"]=df["SYM_One Row Coded Symptoms"].str.replace(', ', ',')
df["AEC_One Row Outcomes"]=df["AEC_One Row Outcomes"].str.replace(', ', ',')
df.reset_index(inplace=True)

In [4]:
df.loc[~df['CI_Gender'].isin(['Female','Male']),'CI_Gender']='Not Available'

In [5]:
df.head()

,RA_Report #,RA_CAERS Created Date,AEC_Event Start Date,PRI_Product Role,PRI_Reported Brand/Product Name,PRI_FDA Industry Code,PRI_FDA Industry Name,CI_Age at Adverse Event,Age in years,CI_Age Unit,Category,CI_Gender,AEC_One Row Outcomes,SYM_One Row Coded Symptoms
0,71436,8/18/2004,4/21/2004,Suspect,OCEAN BREEZE RAW OYSTERS,16,Fishery/Seafood Prod,76.0,760.0,Decade(s),65 and more,Male,"DEATH,VISITED A HEALTH CARE PROVIDER,VISITED A...","DEATH,GASTROENTERITIS VIBRIO"
1,110300,2/18/2009,2/13/2009,Suspect,SIMPY JIF PEANUT BUTTER,23,Nuts/Edible Seed,250.0,250.0,Year(s),65 and more,Female,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),"NAUSEA,VOMITING,ABDOMINAL PAIN"
2,182151,1/26/2015,NaN,Suspect,HYDROXYCUT APPETITE CONTROL POWDER PACKETS,54,Vit/Min/Prot/Unconv Diet(Human/Animal),167.0,167.0,Year(s),65 and more,Female,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),"BACK PAIN,NAUSEA,VOMITING,DIARRHOEA,COLD SWEAT..."
3,182151,1/26/2015,NaN,Suspect,HYDROXYCUT PRO CLINICAL POWDER PACKETS,54,Vit/Min/Prot/Unconv Diet(Human/Animal),167.0,167.0,Year(s),65 and more,Female,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),"BACK PAIN,NAUSEA,VOMITING,DIARRHOEA,COLD SWEAT..."
4,197010,06-07-2016,NaN,Suspect,REDACTED,53,Cosmetics,156.0,156.0,Year(s),65 and more,Female,DEATH,OVARIAN CANCER


In [6]:
wide_df=df.groupby(['RA_Report #']).agg({'PRI_Reported Brand/Product Name': lambda x: x.tolist(),
                                         'SYM_One Row Coded Symptoms': lambda x: x.tolist(),
                                         'AEC_One Row Outcomes': lambda x: x.tolist(),
                                         'CI_Gender': lambda x: list(set(x.tolist())),
                                         'Category': lambda x: list(set(x.tolist())),
                                         'PRI_Product Role': lambda x: list(set(x.tolist())),
                                         'PRI_FDA Industry Name': lambda x: list(set(x.tolist()))
                                         }).rename({'PRI_Reported Brand/Product Name' : 'Product',
                                                    'SYM_One Row Coded Symptoms' : 'Symptoms',
                                                    'AEC_One Row Outcomes' : 'Outcomes',
                                                    'CI_Gender' : 'Gender',
                                                    'Category':'Age_Category',
                                                    'PRI_Product Role':'Product_Role',
                                                   'PRI_FDA Industry Name':'Industry'},axis=1)

In [7]:
def make_set(x):
    result=set()
    for item in x:
        result=set.union(result,set(item.split(',')))
    return list(result)

In [8]:
wide_df[['Product','Symptoms','Outcomes']]=wide_df[['Product','Symptoms','Outcomes']].applymap(make_set)

### Store the cleaned dataset.. No need to preprocess again

In [26]:
wide_df.sample(5)

,Product,Symptoms,Outcomes,Gender,Age_Category,Product_Role,Industry
RA_Report #,,,,,,,
140189,"[HYDROXYCUT REGULAR RAPID RELEASE CAPLETS, HYD...","[EMOTIONAL DISTRESS, RENAL DISORDER, INJURY]",[OTHER SERIOUS (IMPORTANT MEDICAL EVENTS)],[Not Available],[Not defined],[Suspect],[Vit/Min/Prot/Unconv Diet(Human/Animal)]
190937,"[CALCIUM, NUTRITIA NEOCATE JR, VITAMIN D]","[BLOOD ALKALINE PHOSPHATASE INCREASED, BONE DE...",[VISITED A HEALTH CARE PROVIDER],[Male],[Adolescent],"[Suspect, Concomitant]","[Baby Food Prod, Vit/Min/Prot/Unconv Diet(Huma..."
66163,[CHEERIOS CEREAL],[ABDOMINAL PAIN],[NON-SERIOUS INJURIES/ ILLNESS],[Male],[Aged],[Suspect],[Cereal Prep/Breakfast Food]
152084,"[PRESERVISION (MULTIVITAMIN), SPRING VALLELY S...","[HYPOTENSION, DIZZINESS]",[OTHER SERIOUS (IMPORTANT MEDICAL EVENTS)],[Female],[65 and more],"[Suspect, Concomitant]",[Vit/Min/Prot/Unconv Diet(Human/Animal)]
198602,"[ALL DAY ENERGY GREENS, VITAMIN D]","[OESOPHAGEAL PAIN, ABDOMINAL PAIN UPPER]","[OTHER SERIOUS (IMPORTANT MEDICAL EVENTS), VIS...",[Female],[Not defined],"[Suspect, Concomitant]",[Vit/Min/Prot/Unconv Diet(Human/Animal)]


# How to filter columns ?

In [10]:
temp=wide_df
filter_dict={'Product':'JACK 3D','Gender':'Male'}
#Define Column name as key and target_filter_value as value.
#Multiple filter require selection of multiple values from dropdown and is not available unless we need it
for filter_col in filter_dict.keys():
    temp=temp[[filter_dict[filter_col] in item for item in temp[filter_col]]]
temp

,Product,Symptoms,Outcomes,Gender,Age_Category,Product_Role,Industry
RA_Report #,,,,,,,
138210,"[MEGA MEN EXTREME ATHLETE VITAPAK, ANABOLIC HA...","[CHEST PAIN, CARDIAC FLUTTER]",[OTHER SERIOUS (IMPORTANT MEDICAL EVENTS)],[Male],[Adult],"[Suspect, Concomitant]",[Vit/Min/Prot/Unconv Diet(Human/Animal)]
139944,[JACK 3D],[MYOCARDIAL INFARCTION],[HOSPITALIZATION],[Male],[Aged],[Suspect],[Vit/Min/Prot/Unconv Diet(Human/Animal)]
139945,[JACK 3D],[TACHYCARDIA],[VISITED AN ER],[Male],[Adult],[Suspect],[Vit/Min/Prot/Unconv Diet(Human/Animal)]
139946,[JACK 3D],[MYOCARDIAL INFARCTION],"[HOSPITALIZATION, VISITED A HEALTH CARE PROVIDER]",[Male],[Adult],[Suspect],[Vit/Min/Prot/Unconv Diet(Human/Animal)]
142501,"[JACK 3D, ENERGY DRINK]",[LOSS OF CONSCIOUSNESS],[VISITED AN ER],[Male],[Adult],"[Suspect, Concomitant]",[Vit/Min/Prot/Unconv Diet(Human/Animal)]
143955,"[AMPLIFIED WHEYBOLIC EXTREME 60, JACK 3D, OXYE...",[HEPATIC ENZYME INCREASED],"[OTHER SERIOUS (IMPORTANT MEDICAL EVENTS), VIS...",[Male],[Not defined],[Suspect],[Vit/Min/Prot/Unconv Diet(Human/Animal)]
154448,"[TRIPLE STRENGTH FISH OIL, CYTOGAINER, UNKNOWN...","[FATIGUE, SYNCOPE, DYSPNOEA, DIZZINESS, PALPIT...",[OTHER SERIOUS (IMPORTANT MEDICAL EVENTS)],[Male],[Adult],"[Suspect, Concomitant]",[Vit/Min/Prot/Unconv Diet(Human/Animal)]


# Selecting the product

In [11]:
def getkeys(item,df,top_n=15):
    unique_items=df[item].explode().unique()
    if len(unique_items)>top_n:
        count_items=df[item].explode().value_counts()
        count_items=list(count_items[:15].index)
        return count_items
    return list(unique_items)

getkeys(item='Industry',df=wide_df)

['Vit/Min/Prot/Unconv Diet(Human/Animal)',
 'Cosmetics',
 'Nuts/Edible Seed',
 'Vegetables/Vegetable Products',
 'Soft Drink/Water',
 'Bakery Prod/Dough/Mix/Icing',
 'Fishery/Seafood Prod',
 'Fruit/Fruit Prod',
 'Milk/Butter/Dried Milk Prod',
 'Dietary Conv Food/Meal Replacements',
 'Cereal Prep/Breakfast Food',
 'Baby Food Prod',
 'Mult Food Dinner/Grav/Sauce/Special',
 'Snack Food Item',
 'Coffee/Tea']

In [12]:
getkeys(item='Product_Role',df=wide_df)

['Suspect', 'Concomitant']

In [13]:
def make_treemap(df,child1,child2,filter_dict={},child1_top_n=15,child2_top_n=15,sunburst=False):
    '''
    Available options:
    
    Product Product_Role Symptoms Outcomes Gender Age_Category Industry
    '''
    
    start=time.time()
    print('Plotting ',child1," and ",child2)
    
    if not filter_dict:#sneaky way to check if dict is empty
        df_filtered=wide_df
    else:
        #Define Column name as key and target_filter_value as value.
        #Multiple filter require selection of multiple values from dropdown and is not available unless we need it
        df_filtered=wide_df
        for filter_col in filter_dict.keys():
            print('Filtering for... ',filter_col,' = ',filter_dict[filter_col])
            df_filtered=df_filtered[[filter_dict[filter_col] in item for item in df_filtered[filter_col]]]
        # IF no filter => More time to execute
       
    #Improvement Options:
    # 1) if child1=child2 return error? -- It actually gives us a measure of association
    #
    # 2) Instead of top N.... should there be top a to top b ?? .....
    #  ie, Insted of top 1 to 10.... top 3 to 17 ??
    #
    # 3) Instead of taking count.. something like...
    #  count/total_occurance ??
    #
    
    total_rows=len(df_filtered)
    
    child1_keys=getkeys(child1,df=df_filtered,top_n=child1_top_n)
    child2_keys=getkeys(child2,df=df_filtered,top_n=child2_top_n)
    total_child1_keys=len(child1_keys)
    total_child2_keys=len(child2_keys)

    Count_mat = np.zeros([total_child1_keys,total_child2_keys], dtype = int)

    pos_1=0
    pos_2=0
    
    for i in range(0,total_rows):#For every row in dataset 
        for pos_1,child1_item in enumerate(child1_keys):#For every child in top child1_top_n list
            if child1_item in df_filtered[child1].iloc[i]:#If child1 is in child1list
                for pos_2,child2_item in enumerate(child2_keys):#For every child in top child2_top_n list
                    if child2_item in df_filtered[child2].iloc[i]:#If child2 is in child2list
                        Count_mat[pos_1,pos_2]+=1#Value corresponding to child2n1 is increased
    
    df_child2n1Tree=pd.DataFrame(Count_mat, columns=child2_keys)
    df_child2n1Tree['parent']=child1_keys
    
    df_child2n1Tree=df_child2n1Tree.melt(id_vars=["parent"], 
        var_name="id", 
        value_name="Count")
    
    if sunburst:
        fig = px.sunburst(df_child2n1Tree,path=['parent','id'], values='Count')
        fig.update_traces(root_color="lightgrey")
        fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
        fig.show()
    else:
        fig = px.treemap(df_child2n1Tree,path=[px.Constant("Total Cases"),'parent','id'], values='Count')
        fig.update_traces(root_color="lightgrey")
        fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
        fig.show()
    print("..........Executed in ",time.time()-start," Seconds..........\n\n\n")

In [27]:
make_treemap(wide_df,'Symptoms','Gender')

Plotting  Symptoms  and  Gender


..........Executed in  8.49429965019226  Seconds..........





In [15]:
filtered={'Gender':'Male'}
make_treemap(wide_df,'Symptoms','Age_Category',filter_dict=filtered)

Plotting  Symptoms  and  Age_Category
Filtering for...  Gender  =  Male


..........Executed in  3.780921459197998  Seconds..........





In [16]:
#To check if filter worked
filtered={'Gender':'Male'}
make_treemap(wide_df,'Symptoms','Gender',filter_dict=filtered)

Plotting  Symptoms  and  Gender
Filtering for...  Gender  =  Male


..........Executed in  2.3737599849700928  Seconds..........





In [17]:
make_treemap(wide_df,'Symptoms','Gender',sunburst=True)

Plotting  Symptoms  and  Gender


..........Executed in  9.365992784500122  Seconds..........





# All possible treemaps without filter

In [18]:
options=['Product','Product_Role','Symptoms','Outcomes','Gender','Age_Category','Industry']
for i in options:
    for j in options:
        make_treemap(wide_df,child1=i,child2=j)

Plotting  Product  and  Product


..........Executed in  10.185325145721436  Seconds..........



Plotting  Product  and  Product_Role


..........Executed in  7.853242874145508  Seconds..........



Plotting  Product  and  Symptoms


..........Executed in  10.423120498657227  Seconds..........



Plotting  Product  and  Outcomes


..........Executed in  9.616279363632202  Seconds..........



Plotting  Product  and  Gender


...

..........Executed in  14.996678590774536  Seconds..........



Plotting  Industry  and  Outcomes


..........Executed in  13.52681827545166  Seconds..........



Plotting  Industry  and  Gender


..........Executed in  9.239176750183105  Seconds..........



Plotting  Industry  and  Age_Category


..........Executed in  10.982696533203125  Seconds..........



Plotting  Industry  and  Industry


..........Executed in  14.587946653366089  Seconds..........





In [22]:
filtered={'Gender':'Male'}
make_treemap(wide_df,'Outcomes','Industry',filter_dict=filtered)

Plotting  Outcomes  and  Industry
Filtering for...  Gender  =  Male


..........Executed in  4.353031396865845  Seconds..........



